In [1]:
import math
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from contextlib import closing
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
import hashlib

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install -q google-cloud-storage==1.43.0

In [3]:
# Put your bucket name below and make sure you can access it without an error
bucket_name_body = 'project_ir313254724_322769175_body'
full_path = f"gs://{bucket_name_body}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name_body)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [4]:
RE_TOKENIZE = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    return [token.group() for token in RE_TOKENIZE.finditer(text.lower()) if (token.group().strip() and token.group() not in stop_words)]

def word_count(text, id, stem=False):
    ''' Count the frequency of each word in text (tf) that is not included in
    all_stopwords and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
    Text of one document
    id: int
    Document id
    Returns:
    --------
    List of tuples
    A list of (token, (doc_id, tf)) pairs
    for example: [("Anarchism", (12, 5)), ...]
    '''
    global_dict = {}
    all_tokens = tokenize(text)
    tokens = stemming(all_tokens, stem)
    for token in tokens:
      val = global_dict.get(token)
      if val is None:
        global_dict[token] = 1
      else:
        global_dict[token] += 1
    final = [(token, (id, freq)) for token, freq in global_dict.items()]
    return final

'''
sorting the posting list by doc_id. we will apply this function with map reduce
and it will be ended as dict of token: posting_list key-value.
'''
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
    A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
    A sorted posting list.
    '''
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])  # Sorting by wiki_id
    return sorted_pl

'''
idf added to this dict. this will be dict of {token: [df, idf]} and it will help to calculate the BM25.
'''
def calculate_df(postings, N):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
    An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
    An RDD where each element is a (token, df) pair.
    '''
    return postings.map(lambda x: (x[0], (len(x[1]), math.log(N / len(x[1])))))

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

def token2bucket_id(token):
    NUM_BUCKETS = 124
    return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of token2bucket
    above. Writing to disk should use the function  write_a_posting_list, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
    An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
    An RDD where each item is a posting locations dictionary for a bucket. The
    posting locations maintain a list for each word of file locations and
    offsets its posting list was written to. See write_a_posting_list for
    more details.
    '''
    #create rdd of (bucket_id,(w, posting_list))
    bucket_postrdd = postings.map(lambda post_rdd: (token2bucket_id(post_rdd[0]),post_rdd)).groupByKey()
    final = bucket_postrdd.map(lambda buck: InvertedIndex.write_a_posting_list(buck,"body_posting/",bucket_name_body))
    return final


def stemming(tokens, stem=False):
    length_list = len(tokens)
    if stem:
      stemmer = PorterStemmer()
      index = 0
      while index < length_list:
          tokens[index] = stemmer.stem(tokens[index])
          index += 1
    return tokens

In [5]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs_text = parquetFile.select("text", "id").rdd

In [6]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [7]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import *

In [8]:
stop_words1 = set(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

stop_words = stop_words1.union(corpus_stopwords)

In [9]:
# Tokenize each document and count the number of tokens
doc_token_counts_body = doc_text_pairs_text.map(lambda x: (x[1], len(tokenize(x[0]))))

# Calculate the total number of documents and the total number of tokens
total_documents_body = doc_text_pairs_text.count()
total_tokens_body = doc_token_counts_body.map(lambda x: x[1]).sum()

# Calculate the average number of tokens per document
avg_tokens_per_doc_body = total_tokens_body / total_documents_body

In [10]:
# word counts map
flag_body = True
word_counts_body = doc_text_pairs_text.flatMap(lambda x: word_count(x[0], x[1], flag_body))
postings_body = word_counts_body.groupByKey().mapValues(lambda x: reduce_word_counts(list(x)))
postings_filtered = postings_body.filter(lambda x: len(x[1]) > 50)
w2df_body = calculate_df(postings_filtered, total_documents_body)
w2df_dict_body = w2df_body.collectAsMap()

# partition posting lists and write out
_body = partition_postings_and_write(postings_filtered).collect()

In [11]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name_body, prefix='body_posting'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v)

In [12]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_dict_body
inverted_body.stem = flag_body

'''
insert important values into InvertedIndex object.
'''
inverted_body.N = total_documents_body
inverted_body.avg_doclen = avg_tokens_per_doc_body
inverted_body.docs_len = doc_token_counts_body.collectAsMap() # Dict of doc_id: len_doc.

In [13]:
inverted_body.write_index('.', 'body_index')
index_src_body = "body_index.pkl"
index_dst_body = f'gs://{bucket_name_body}/body_posting/{index_src_body}'
!gsutil cp $index_src_body $index_dst_body
inverted_body.write_index('.', 'body_index')

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 65.4 MiB/ 65.4 MiB]                                                
Operation completed over 1 objects/65.4 MiB.                                     
